<a href="https://colab.research.google.com/github/thkted/DeepLearning_Study/blob/main/%EB%AA%A8%EB%91%90%EC%9D%98%EB%94%A5%EB%9F%AC%EB%8B%9D/Lab6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow
!pip install tensorflow==1.15

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 512kB 53.6MB/s 
     |████████████████████████████████| 3.8MB 46.5MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=6003a006677359d0addb2d9582a6f755043891400fa82a635f1a7562896309ba
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successf

In [2]:
import gym
import tensorflow as tf
import numpy as np

In [3]:
env = gym.make('CartPole-v0')
env.reset()

array([ 0.04145423, -0.04985076, -0.04517819, -0.02337134])

In [4]:
random_episodes = 0
reward_sum = 0

lr = 0.1
input_size = env.observation_space.shape[0] # 4
output_size = env.action_space.n # 2

In [5]:
X = tf.placeholder(tf.float32, [None, input_size], name='input_x') # [1,4]
W1 = tf.get_variable('W1', shape=[input_size, output_size],
                     initializer=tf.contrib.layers.xavier_initializer())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
Qpred = tf.matmul(X, W1)

Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

loss = tf.reduce_sum(tf.square(Y - Qpred))
train = tf.train.AdamOptimizer(learning_rate = lr).minimize(loss)

In [7]:
num_episodes = 2000
dis = 0.9
rList = []

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(num_episodes):
  e = 1. / ( (i /10) + 1)
  rAll = 0
  step_count = 0
  s = env.reset()
  done = False

  while not done:
    step_count += 1
    x = np.reshape(s, [1, input_size]) # [1,4]

    Qs = sess.run(Qpred, feed_dict={X: x})

    if np.random.random(1) < e:
      a = env.action_space.sample()
    else:
      a = np.argmax(Qs)
    
    s1, reward, done, _ = env.step(a)

    if done:
      Qs[0, a] = -100
    else:
      x1 = np.reshape(s1, [1, input_size])
      Qs1 = sess.run(Qpred, feed_dict={X:x1})
      Qs[0,a] = reward + dis * np.argmax(Qs1)
    
    sess.run(train, feed_dict={X:x, Y:Qs})
    s = s1
  
  rList.append(step_count)
  print('Episode: {} / steps: {}'.format(i, step_count))

  if len(rList) > 10 and np.mean(rList[-10:]) > 500:
    break

Episode: 0 / steps: 35
Episode: 1 / steps: 13
Episode: 2 / steps: 13
Episode: 3 / steps: 10
Episode: 4 / steps: 20
Episode: 5 / steps: 16
Episode: 6 / steps: 13
Episode: 7 / steps: 23
Episode: 8 / steps: 71
Episode: 9 / steps: 18
Episode: 10 / steps: 48
Episode: 11 / steps: 96
Episode: 12 / steps: 18
Episode: 13 / steps: 30
Episode: 14 / steps: 35
Episode: 15 / steps: 69
Episode: 16 / steps: 53
Episode: 17 / steps: 75
Episode: 18 / steps: 54
Episode: 19 / steps: 64
Episode: 20 / steps: 73
Episode: 21 / steps: 60
Episode: 22 / steps: 36
Episode: 23 / steps: 42
Episode: 24 / steps: 26
Episode: 25 / steps: 30
Episode: 26 / steps: 80
Episode: 27 / steps: 19
Episode: 28 / steps: 41
Episode: 29 / steps: 37
Episode: 30 / steps: 29
Episode: 31 / steps: 31
Episode: 32 / steps: 29
Episode: 33 / steps: 30
Episode: 34 / steps: 37
Episode: 35 / steps: 36
Episode: 36 / steps: 22
Episode: 37 / steps: 24
Episode: 38 / steps: 28
Episode: 39 / steps: 38
Episode: 40 / steps: 32
Episode: 41 / steps: 37
Ep